<a href="https://colab.research.google.com/github/repoocsov/DS-Unit-2-Applied-Modeling/blob/master/module1-define-ml-problems/LS_DS_231_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Define ML problems

You will use your portfolio project dataset for all assignments this sprint.

## assignment

Complete these tasks for your project, and document your decisions.

- [x] Choose your target. Which column in your tabular dataset will you predict?
- [x] Is your problem regression or classification?
- [x] How is your target distributed?
    - Classification: How many classes? Are the classes imbalanced?
    - Regression: Is the target right-skewed? If so, you may want to log transform the target.
- [x] Choose your evaluation metric(s).
    - Classification: Is your majority class frequency >= 50% and < 70% ? If so, you can just use accuracy if you want. Outside that range, accuracy could be misleading. What evaluation metric will you choose, in addition to or instead of accuracy?
    - Regression: Will you use mean absolute error, root mean squared error, R^2, or other regression metrics?
- [x] Choose which observations you will use to train, validate, and test your model.
    - Are some observations outliers? Will you exclude them?
    - Will you do a random split or a time-based split?
- [x] Begin to clean and explore your data.
- [x] Begin to choose which features, if any, to exclude. Would some features "leak" future information?

If you haven't found a dataset yet, do that today. [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2) and choose your dataset.

Some students worry, ***what if my model isn't “good”?*** Then, [produce a detailed tribute to your wrongness. That is science!](https://twitter.com/nathanwpyle/status/1176860147223867393)

In [0]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/dlau/sr-data/master/dump-09-27-2014.csv')

In [8]:
# NUMERIC
df.describe().columns

Index(['timestamp', 'price_usd', 'vendor_id', 'product_rating'], dtype='object')

In [9]:
# NON-NUMERIC
df.describe(exclude="number").columns

Index(['drug_name', 'ships_from', 'ships_to', 'vendor_rating'], dtype='object')

In [11]:
# TARGET
df['drug_name'].value_counts(normalize=True)

mdma           0.368259
weed           0.212070
lsd            0.145851
cocaine        0.132719
amphetamine    0.060073
heroin         0.041911
ketamine       0.018720
opium          0.014809
mescaline      0.005588
Name: drug_name, dtype: float64

In [12]:
print(1/9)

# Classes are pretty imbalanced

0.1111111111111111


In [0]:
# Majority class is 37%
# Accuracy score could be misleading on it's own as it's it's not 50-70%
"""
‘accuracy’
metrics.accuracy_score

‘balanced_accuracy’
metrics.balanced_accuracy_score
"""

In [13]:
# Splitting.
# Split based on the target (stratify)
# product_rating is highly skewed. Maybe encode them using bins
df.describe()

,timestamp,price_usd,vendor_id,product_rating
count,3.579000e+03,3.579000e+03,3579.000000,3579.000000
mean,1.411854e+12,3.146025e+03,152.122381,28.751886
std,1.142783e+06,7.045842e+04,98.879783,109.254532
min,1.411853e+12,0.000000e+00,0.000000,0.000000
25%,1.411853e+12,8.718774e+01,67.000000,0.000000
50%,1.411854e+12,2.547720e+02,143.000000,1.000000
75%,1.411856e+12,9.418553e+02,229.000000,11.000000
max,1.411856e+12,4.132407e+06,366.000000,2199.000000


In [0]:
"""
  I think I'm going to use a different dataset
"""